# Use Customer_Segmentation Notebook Template

In [1]:
!pip install --q --upgrade pip

In [2]:
!pip install --q seaborn

# Kernel Restart here

In [3]:
import numpy as np # linear algebra
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix

from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

import warnings
warnings.filterwarnings('ignore')

In [4]:
table_name = 'CUSTOMER_DATA_INSIGHT_TRAIN_OUTPUT'

sf_df = my_session.sql("select * from {}".format(table_name))
train = sf_df.to_pandas()

In [5]:
table_name = 'CUSTOMER_DATA_INSIGHT_TEST_OUTPUT'

sf_df = my_session.sql("select * from {}".format(table_name))
test = sf_df.to_pandas()

In [8]:
test.shape

(40000, 32)

In [10]:
train = train[['CUSTOMER_ID', 'DATE', 'AGE', 'SEX', 'EDUCATION_LEVEL',
       'EMPLOYMENT_STATUS', 'HOBBIES', 'MARITAL_STATUS', 'DEPENDENTS',
       'REGISTRATION_DATE', 'BANK_ACCOUNT_TYPE', 'BALANCE',
       'BALANCE_FREQUENCY', 'CREDIT_LIMIT', 'CASH_ADVANCE',
       'CASH_ADVANCE_FREQUENCY', 'CASH_ADVANCE_TRX', 'PURCHASES',
       'ONEOFF_PURCHASES', 'INSTALLMENTS_PURCHASES', 'PURCHASES_FREQUENCY',
       'ONEOFF_PURCHASES_FREQUENCY', 'PURCHASES_INSTALLMENTS_FREQUENCY',
       'PURCHASES_TRX', 'PAYMENTS', 'MINIMUM_PAYMENTS', 'PRC_FULL_PAYMENT',
       'CLUSTER', 'CLUSTER_TYPE']]

In [11]:
test = test[['CUSTOMER_ID', 'DATE', 'AGE', 'SEX', 'EDUCATION_LEVEL',
       'EMPLOYMENT_STATUS', 'HOBBIES', 'MARITAL_STATUS', 'DEPENDENTS',
       'REGISTRATION_DATE', 'BANK_ACCOUNT_TYPE', 'BALANCE',
       'BALANCE_FREQUENCY', 'CREDIT_LIMIT', 'CASH_ADVANCE',
       'CASH_ADVANCE_FREQUENCY', 'CASH_ADVANCE_TRX', 'PURCHASES',
       'ONEOFF_PURCHASES', 'INSTALLMENTS_PURCHASES', 'PURCHASES_FREQUENCY',
       'ONEOFF_PURCHASES_FREQUENCY', 'PURCHASES_INSTALLMENTS_FREQUENCY',
       'PURCHASES_TRX', 'PAYMENTS', 'MINIMUM_PAYMENTS', 'PRC_FULL_PAYMENT',
       'CLUSTER', 'CLUSTER_TYPE']]

In [12]:
df = pd.concat([train, test], axis=0)

In [13]:
df.shape

(120000, 29)

In [14]:
df.tail()

,CUSTOMER_ID,DATE,AGE,SEX,EDUCATION_LEVEL,EMPLOYMENT_STATUS,HOBBIES,MARITAL_STATUS,DEPENDENTS,REGISTRATION_DATE,...,INSTALLMENTS_PURCHASES,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,PURCHASES_TRX,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,CLUSTER,CLUSTER_TYPE
39995,60e2911b-6a1c-4d35-bd6f-700a96055b7d,2022-12-01,32,FEMALE,JD,Employed,reading,Single,One Dependent,2020-02-12,...,0.0000,Very low,Very low,Very low,0.00,1911.368074,381.694812,0.00,2,Cash Advance Users
39996,96454f77-584a-442b-8f70-bc38e8630908,2024-02-01,25,FEMALE,JD,Unemployed,reading,Single,One Dependent,2017-04-12,...,109.2648,Low,Very low,Low,4.32,0.000000,0.000000,0.00,0,Conservative Spenders
39997,1429938e-ab3d-4d43-b0a4-e0d3331501a6,2024-05-01,33,FEMALE,JD,Unemployed,hiking,Single,One Dependent,2019-04-12,...,269.8800,Very high,Very low,Very high,26.00,4086.277963,1723.818986,0.50,0,Conservative Spenders
39998,558c05aa-e580-47a3-8257-c3b9dbbcca6f,2023-09-01,33,FEMALE,MD,Unemployed,board-games,Single,One Dependent,2005-11-12,...,250.9884,Very high,Very low,Very high,28.08,4372.317420,1663.485321,0.54,0,Conservative Spenders
39999,1e307f04-29b5-497d-b47f-e1aa0ee43fa0,2023-11-01,41,MALE,JD,Unemployed,base-jumping,Single,Two Dependents,2021-09-12,...,2548.1092,Very high,Very low,Very high,46.20,2666.034909,161.259890,0.70,3,Active Credit Users


In [17]:
to_drop = ['CUSTOMER_ID','DATE','AGE','SEX','EDUCATION_LEVEL','EMPLOYMENT_STATUS','HOBBIES','MARITAL_STATUS','BANK_ACCOUNT_TYPE',
 'DEPENDENTS','REGISTRATION_DATE','PURCHASES_FREQUENCY','ONEOFF_PURCHASES_FREQUENCY','PURCHASES_INSTALLMENTS_FREQUENCY','BALANCE_FREQUENCY','CLUSTER_TYPE']

In [18]:
df.drop(to_drop, axis= 1, inplace= True)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 120000 entries, 0 to 39999
Data columns (total 13 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   BALANCE                 120000 non-null  float64
 1   CREDIT_LIMIT            120000 non-null  float64
 2   CASH_ADVANCE            120000 non-null  float64
 3   CASH_ADVANCE_FREQUENCY  120000 non-null  float64
 4   CASH_ADVANCE_TRX        120000 non-null  float64
 5   PURCHASES               120000 non-null  float64
 6   ONEOFF_PURCHASES        120000 non-null  float64
 7   INSTALLMENTS_PURCHASES  120000 non-null  float64
 8   PURCHASES_TRX           120000 non-null  float64
 9   PAYMENTS                120000 non-null  float64
 10  MINIMUM_PAYMENTS        120000 non-null  float64
 11  PRC_FULL_PAYMENT        120000 non-null  float64
 12  CLUSTER                 120000 non-null  int8   
dtypes: float64(12), int8(1)
memory usage: 12.0 MB


In [20]:
X = df.drop(['CLUSTER'],axis = 1 )
y = df[['CLUSTER']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3,random_state = 101)

In [21]:
X_train.shape, X_test.shape

((84000, 12), (36000, 12))

# Decision Tree Classifier

In [22]:
#Training our algorithm
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
clf_prediction = clf.predict(X_test)

In [23]:
#Checking our prediction accuracy score
print(confusion_matrix(y_test, clf_prediction))

[[24975     0   124   108]
 [    0   288     0    12]
 [  121     0  5845    21]
 [  115     9    23  4359]]


In [24]:
print(classification_report(y_test, clf_prediction))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     25207
           1       0.97      0.96      0.96       300
           2       0.98      0.98      0.98      5987
           3       0.97      0.97      0.97      4506

    accuracy                           0.99     36000
   macro avg       0.98      0.97      0.97     36000
weighted avg       0.99      0.99      0.99     36000



In [25]:
clf.feature_importances_

array([0.08211526, 0.03631013, 0.28738298, 0.03441241, 0.10608844,
       0.34331389, 0.00521996, 0.01469608, 0.04108736, 0.0238691 ,
       0.01311726, 0.01238713])

In [26]:
pd.Series(clf.feature_importances_,index=X_train.columns).sort_values(ascending=False)

PURCHASES                 0.343314
CASH_ADVANCE              0.287383
CASH_ADVANCE_TRX          0.106088
BALANCE                   0.082115
PURCHASES_TRX             0.041087
CREDIT_LIMIT              0.036310
CASH_ADVANCE_FREQUENCY    0.034412
PAYMENTS                  0.023869
INSTALLMENTS_PURCHASES    0.014696
MINIMUM_PAYMENTS          0.013117
PRC_FULL_PAYMENT          0.012387
ONEOFF_PURCHASES          0.005220
dtype: float64

# Model registration Code

In [27]:
type(X_train),type(X_test),type(y_train), type(y_test), type(clf_prediction)

(pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 numpy.ndarray)

In [45]:
## registering the model in Fosfor.
model_reg = register_model(clf,
               score, 
               name="Customer_Segmentation_Dtree_Cluster_Classifier", 
               description="Customer_Segmentation_Dtree_Cluster_Classifier",
               flavour=MLModelFlavours.sklearn,
               model_type="classification",
               init_script="\\n pip install fosforml \\n pip install fosforio[snowflake] \\n pip install seaborn \\n pip install snowflake-connector-python[pandas]",
               y_true=y_test,
               y_pred=clf_prediction,
               prob=clf_prediction,
               features=X_train.columns,
               input_type="json", 
               explain_ai=True,
               x_train=X_train, 
               x_test=X_test, 
               y_train=y_train,
               y_test=y_test,
               feature_names=X_train.columns.tolist(),
               original_features=X_train.columns.tolist(),
               feature_ids=X_train.columns,
               kyd=True, kyd_score = True)

Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


# Another Version

In [46]:
clf_prediction = clf.predict(X_train)

In [47]:
## registering the model in Fosfor.
model_reg = register_model(clf,
               score, 
               name="Customer_Segmentation_Dtree_Cluster_Classifier", 
               description="Customer_Segmentation_Dtree_Cluster_Classifier",
               flavour=MLModelFlavours.sklearn,
               model_type="classification",
               init_script="\\n pip install fosforml \\n pip install fosforio[snowflake] \\n pip install seaborn \\n pip install snowflake-connector-python[pandas]",
               y_true=y_train,
               y_pred=clf_prediction,
               prob=clf_prediction,
               features=X_train.columns,
               input_type="json", 
               explain_ai=True,
               x_train=X_test, 
               x_test=X_train, 
               y_train=y_test,
               y_test=y_train,
               feature_names=X_train.columns.tolist(),
               original_features=X_train.columns.tolist(),
               feature_ids=X_train.columns,
               kyd=True, kyd_score = True)

Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


# Write back the output to Snowflake

In [49]:
df = pd.read_csv('/data/Output/kmeans_clustered_customer_Data.csv')

In [52]:
df.head()

,CUSTOMER_ID,DATE,AGE,SEX,EDUCATION_LEVEL,EMPLOYMENT_STATUS,HOBBIES,MARITAL_STATUS,DEPENDENTS,REGISTRATION_DATE,...,INSTALLMENTS_PURCHASES,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,PURCHASES_TRX,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,Cluster,Cluster_Type
0,PNC32940ed8-682f-4052-92b2-cfdc7f41e,2023-12-01,49,FEMALE,Undisclosed,Employed,Undisclosed,Married,Two Dependents,2004-11-12,...,0.0,Very low,Very low,Very low,0.000,3774.789989,1007.999804,0.214444,2,Cash Advance Users
1,PNCe415caf5-6212-4f50-a062-c12bf1eea,2024-01-01,34,MALE,Undisclosed,Unemployed,Undisclosed,Married,No Dependents,2023-07-12,...,0.0,Very high,Very high,Very low,11.400,665.611414,589.647700,0.000000,0,Conservative Spenders
2,PNC1ef5b665-11ae-40d7-b364-be8fa02b9,2023-02-01,34,MALE,Undisclosed,Unemployed,Undisclosed,Married,No Dependents,2021-11-12,...,0.0,Very high,Very high,Very low,10.920,653.170079,570.829156,0.000000,0,Conservative Spenders
3,PNC436a6069-0e61-47f2-96e8-d11c19e4b,2024-04-01,29,MALE,Undisclosed,Unemployed,Undisclosed,Single,One Dependent,2019-07-12,...,0.0,Very low,Very low,Very low,0.965,0.000000,0.000000,0.000000,0,Conservative Spenders
4,PNCaaa1c8bb-7289-4c25-a05e-efe26e316,2024-02-01,29,MALE,Undisclosed,Unemployed,Undisclosed,Single,One Dependent,2022-09-12,...,0.0,Very low,Very low,Very low,1.060,0.000000,0.000000,0.000000,0,Conservative Spenders


In [55]:
df['PREDICTION'] = clf.predict(df[X_train.columns])

In [56]:
print(classification_report(df['Cluster'], df['PREDICTION']))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85997
           1       0.99      0.99      0.99      1062
           2       0.99      0.99      0.99     20461
           3       0.99      0.99      0.99     15362

    accuracy                           1.00    122882
   macro avg       0.99      0.99      0.99    122882
weighted avg       1.00      1.00      1.00    122882



In [57]:
from snowflake.snowpark import Session
import warnings; warnings.simplefilter('ignore')
import configparser

In [58]:
#Import all snowflake connection details from template variables.
db_user = 'MANISH'
db_password = 'Password@2023'
db_account = 'ug94937.us-east4.gcp'
db_database =  'FDC_Insurance'
db_role = 'MANISH'
db_warehouse = 'FOSFOR_SOLUTIONS_WH'
db_schema = 'INS_CUST_SEGMT_SCHEMA'

In [59]:
from snowflake.snowpark.session import Session
connection_params = {
    'user': db_user,
    'password': db_password,
    'account': db_account,
    'warehouse': db_warehouse,
    'database': db_database,
    'schema': db_schema,
    'role': db_role
}
session1 = Session.builder.configs(connection_params).create()

In [60]:
df.shape

(122882, 30)

In [61]:
df_train = df.head(80000)
df_test = df.tail(40000)

In [62]:
df_train_sf=session1.createDataFrame(
        df_train.values.tolist(),
        schema=df_train.columns.tolist())
df_train_sf.write.mode("overwrite").save_as_table("FDC_Insurance.INS_CUST_SEGMT_SCHEMA.B")

In [63]:
df_test_sf=session1.createDataFrame(
        df_test.values.tolist(),
        schema=df_test.columns.tolist())
df_test_sf.write.mode("overwrite").save_as_table("FDC_Insurance.INS_CUST_SEGMT_SCHEMA.CUSTOMER_DATA_INSIGHT_TEST_OUTPUT")

# Alter and Update Column REPORT_DATE
###### ALTER TABLE FDC_INSURANCE.INS_CUST_SEGMT_SCHEMA.CUSTOMER_DATA_INSIGHT_TRAIN_OUTPUT ADD COLUMN REPORT_DATE DATE

###### UPDATE FDC_INSURANCE.INS_CUST_SEGMT_SCHEMA.CUSTOMER_DATA_INSIGHT_TRAIN_OUTPUT SET REPORT_DATE = TO_DATE(DATE)

###### ALTER TABLE FDC_INSURANCE.INS_CUST_SEGMT_SCHEMA.CUSTOMER_DATA_INSIGHT_TEST_OUTPUT ADD COLUMN REPORT_DATE DATE

###### UPDATE FDC_INSURANCE.INS_CUST_SEGMT_SCHEMA.CUSTOMER_DATA_INSIGHT_TEST_OUTPUT SET REPORT_DATE = TO_DATE(DATE)

# Alter and Add CLUSTER_ID column for Decision designer setup

ALTER TABLE FDC_INSURANCE.INS_CUST_SEGMT_SCHEMA.CUSTOMER_DATA_INSIGHT_TRAIN_OUTPUT ADD COLUMN CLUSTER_ID VARCHAR

ALTER TABLE FDC_INSURANCE.INS_CUST_SEGMT_SCHEMA.CUSTOMER_DATA_INSIGHT_TEST_OUTPUT ADD COLUMN CLUSTER_ID VARCHAR

UPDATE FDC_INSURANCE.INS_CUST_SEGMT_SCHEMA.CUSTOMER_DATA_INSIGHT_TRAIN_OUTPUT
SET CLUSTER_ID = CASE
    WHEN (CLUSTER = 0) THEN 'CLUSTER_0'
    WHEN (CLUSTER = 1) THEN 'CLUSTER_1'
    WHEN (CLUSTER = 2) THEN 'CLUSTER_2'
    ELSE 'CLUSTER_3'
END


UPDATE FDC_INSURANCE.INS_CUST_SEGMT_SCHEMA.CUSTOMER_DATA_INSIGHT_TEST_OUTPUT
SET CLUSTER_ID = CASE
    WHEN (CLUSTER = 0) THEN 'CLUSTER_0'
    WHEN (CLUSTER = 1) THEN 'CLUSTER_1'
    WHEN (CLUSTER = 2) THEN 'CLUSTER_2'
    ELSE 'CLUSTER_3'
END


# Prepare Input data to Run Sample Multi Classification experiments for demo

In [65]:
experiment_feature = ['AGE', 'SEX', 'EDUCATION_LEVEL',
       'EMPLOYMENT_STATUS', 'HOBBIES', 'MARITAL_STATUS', 'DEPENDENTS','BANK_ACCOUNT_TYPE', 'BALANCE',
       'BALANCE_FREQUENCY', 'CREDIT_LIMIT', 'CASH_ADVANCE','CASH_ADVANCE_FREQUENCY', 'CASH_ADVANCE_TRX', 'PURCHASES',
       'ONEOFF_PURCHASES', 'INSTALLMENTS_PURCHASES', 'PURCHASES_FREQUENCY','ONEOFF_PURCHASES_FREQUENCY', 'PURCHASES_INSTALLMENTS_FREQUENCY',
       'PURCHASES_TRX', 'PAYMENTS', 'MINIMUM_PAYMENTS', 'PRC_FULL_PAYMENT','Cluster']

In [67]:
df[experiment_feature].to_csv('/data/Output/kmeans_clustered_experiment_input.csv',index=False)